## IMPORTAR LAS LIBRERIAS


In [1]:
import numpy as np, os, time, random, math
from IPython.display import clear_output
from collections import deque
import time

## TABLERO DE JUEGO

In [2]:
tablero = np.zeros((10,10),dtype = int)
posicion_actual_gato = [1,1]
posicion_actual_raton = [8,8]

## DISTANCIA DE MANHHATTAN


In [3]:
def calcular_distancia_manhhattan(posicion_posible_gato, posicion_actual_raton):
    coordenadas_gato = np.array(posicion_posible_gato)
    coordenadas_raton = np.array(posicion_actual_raton)
    distancia = np.sum(np.abs(coordenadas_gato - coordenadas_raton))
    return(distancia)

In [4]:
#otra forma
# def funcion_de_evaluacion(posicion_gato, posicion_raton):
#     distancia = math.sqrt((posicion_gato[0] - posicion_raton[0])**2 + (posicion_gato[1] - posicion_raton[1])**2)

## FUNCION PARA VER LOS MOVIMIENTOS QUE PUEDE HACER EL RATON


In [5]:
def movimientos_posibles_raton(tablero_actual, posicion_actual):
    direcciones_disponibles = [
        (1,0), #x aumenta en 1, y se queda igual (se mueve a la derecha)
        (-1,0), #x disminuye en 1, y se queda igual (se mueve a la izquierda)
        (0,1), #y aumenta en 1, x se queda igual (se mueve hacia arriba)
        (0,-1), #y disminuye en 1, x se queda igual (se mueve hacia abajo)
        (1,1), #x e y aumentan en 1 (se mueve ariiba a la derecha)
        (1,-1), # x aumenta 1 e y disminuye 1 (se mueve abajo a la derecha)
        (-1,-1), # x e y disminuyen en 1, (se mueve abajo a la izquierda)
        (-1,1) # x disminuye en 1 e y aumenta en 1 (se mueve arriba a la izquerda)
    ]
    

    lista_de_movimientos_disponibles = []
    
    for dx, dy in direcciones_disponibles:
        posible_movimiento_x = posicion_actual[0] + dx
        posible_movimiento_y = posicion_actual[1] + dy
        if posible_movimiento_x >= 0:
            if posible_movimiento_x < 10:
                if posible_movimiento_y >= 0:
                    if posible_movimiento_y < 10:
                        if tablero_actual[ posible_movimiento_x , posible_movimiento_y ] != 9:
                            lista_de_movimientos_disponibles.append((posible_movimiento_x, posible_movimiento_y))
    return(lista_de_movimientos_disponibles)


## FUNCION PARA VER LOS MOVIMIENTOS QUE PUEDE HACER EL GATO


In [6]:
def movimientos_posibles_gato(tablero, posicion_actual_gato):
    direcciones_disponibles = [
        (2,0),
        (1,0),
        (-2,0),
        (-1,0),
        (0,2),
        (0,1),
        (0,-2),
        (0,-1)
    ]
    
    lista_de_direcciones_disponibles = []
    for dx, dy in direcciones_disponibles:
        posible_movimiento_x = posicion_actual_gato[0] + dx
        posible_movimiento_y = posicion_actual_gato[1] + dy

        
            

        if posible_movimiento_x >= 0:
            if posible_movimiento_x < 10:
                if posible_movimiento_y >= 0:
                    if posible_movimiento_y < 10:
                        if tablero[ posible_movimiento_x , posible_movimiento_y ] != 9:
                            if abs(dx) == 2 or abs(dy) == 2:
                                intermedio_x = posicion_actual_gato[0] + (dx // 2)
                                intermedio_y = posicion_actual_gato[1] + (dy // 2)
                                if tablero[intermedio_x, intermedio_y] == 9:
                                    continue
                            
                            lista_de_direcciones_disponibles.append((posible_movimiento_x, posible_movimiento_y))
    return lista_de_direcciones_disponibles

## GATO QUE PERSIGUE LINEALMENTE

In [7]:
def gato_persigue_linealmente(tablero, posicion_actual_gato, posicion_actual_raton):
    movimientos_posibles = movimientos_posibles_gato(tablero, posicion_actual_gato)
    mejor_movimiento = posicion_actual_gato
    mejor_distancia = float('inf')
    for movimiento in movimientos_posibles:

        distancia = calcular_distancia_manhhattan(movimiento, posicion_actual_raton)
        if distancia < mejor_distancia:
            mejor_distancia = distancia
            mejor_movimiento = movimiento

    return mejor_movimiento
        
    
    

## RATON QUE SE MUEVE ALEATORIAMENTE


In [8]:
def movimiento_aleatorio_raton(tablero, posicion_actual_raton):
    movimientos_posibles = movimientos_posibles_raton(tablero,posicion_actual_raton)
    movimiento_elegido = random.choice(movimientos_posibles)
    return(movimiento_elegido[0], movimiento_elegido[1])

## PARA HALLAR EL CAMINO

In [9]:
def movimientos_de_busqueda_posibles(pos_actual, tablero):
    movimientos_disponibles = [(1,0),(-1,0),(0,1),(0,-1)]
    movimientos_posibles = []
    for mx, my in movimientos_disponibles:
        posible_x = pos_actual[0] + mx
        posible_y = pos_actual[1] + my
        if tablero[posible_x, posible_y] != 9:
            movimientos_posibles.append((posible_x, posible_y))
    return movimientos_posibles

## DISTANCIA BFS


In [10]:
def bfs(inicio, objetivo, tablero):
    ## iniciamos una cola que contenga las posiciones y la distancia recorrida hasta ese momento
    cola = deque([(inicio, 0)])

    #guardamos lo que ya se visito para evitar volver por ahi
    visitadas = {inicio}
    distancia = 0
    while cola:
        (coordenadas, distancia) = cola.popleft()
        if coordenadas == objetivo:

            return distancia
            
        else:
            movimientos_disponibles = movimientos_de_busqueda_posibles(coordenadas, tablero)
            for mx, my in movimientos_disponibles:
                nuevo_movimiento = (mx, my)
                if nuevo_movimiento in visitadas:
                    continue
                else:
                    visitadas.add(nuevo_movimiento)
                    nueva_distancia = distancia + 1
                    cola.append((nuevo_movimiento, nueva_distancia))

                



## MINIMAX


In [11]:
def minimax(tablero, profundidad, es_turno_del_gato, posicion_gato, posicion_raton):
    
    if posicion_gato == posicion_raton:
        return -10000
    if profundidad == 0:
        funcion_de_evaluacion = bfs(posicion_gato, posicion_raton, tablero)
        return funcion_de_evaluacion
    if es_turno_del_gato: #jugador q minimiza
        mejor_valor = float('inf')
        posiciones_posibles_gato = movimientos_posibles_gato(tablero, posicion_gato)
        for posible_posicion_gato in posiciones_posibles_gato:
            valor = minimax(
                tablero,
                profundidad - 1,
                False,
                posible_posicion_gato,
                posicion_raton
            )
            mejor_valor = min(mejor_valor, valor)
        return mejor_valor
    else: #jugador que maximiza
        mejor_valor = float('-inf')
        posiciones_posibles = movimientos_posibles_raton(tablero, posicion_raton)
        for posible_posicion_raton in posiciones_posibles:
            valor = minimax(
                tablero,
                profundidad -1,
                True,
                posicion_gato,
                posible_posicion_raton,
                
            )
            mejor_valor = max(mejor_valor, valor)
        return mejor_valor


## MEJOR MOVIMIENTO PARA EL GATO

In [12]:
def mejor_movimiento_gato(tablero, pos_gato, pos_raton):
    direcciones_disponibles = movimientos_posibles_gato(tablero, pos_gato)
    mejor_puntaje = float('inf')
    posicion_elegida = []
    for posicion in direcciones_disponibles:
        puntaje = minimax(tablero, 4, True, posicion, pos_raton)
        if mejor_puntaje > puntaje:
            mejor_puntaje = puntaje
            posicion_elegida = posicion
    return posicion_elegida

    


In [13]:
def mejor_movimiento_raton(tablero, pos_gato, pos_raton):
    direcciones_disponibles = movimientos_posibles_raton(tablero, pos_raton)
    mejor_puntaje = float('-inf')
    posicion_elegida = []
    for posicion in direcciones_disponibles:
        puntaje = minimax(tablero, 4, False, pos_gato, posicion,)
        if mejor_puntaje < puntaje:
            mejor_puntaje = puntaje
            posicion_elegida = posicion
    return posicion_elegida

## MODO DE JUEGO IA BASICA


In [14]:
while True:
    #limpiamos la consola, pero solo anda para python puro
    # os.system(
    #     'cls' if os.name == 'nt'
    #     else 'clear'
    # )
    clear_output(wait=True)
    #definimos los bordes del tablero
    #primero llenamos de ceros, es algo redundante, pero no esta de mas
    tablero[:,:] = 0
    tablero[0,:] = 9
    tablero[9,:] = 9
    tablero[:,0] = 9
    tablero[:,9] = 9

    #ponemos el laberinto correspondiente

    paredes_internas = [(4,1), (2,2), (2,4), (2,6), (3,2), (3,6), (3,7), (4,4), (4,5), (5,2), (5,3), (5,7), (6,5), (7,1), (7,3), (7,7), (8,5)]

    for px, py in paredes_internas:
        tablero[px,py] = 9

    #definimos al gato como un 2 en el tablero y al raton como un 1

    tablero[ posicion_actual_raton[0] , posicion_actual_raton[1] ] = 1

    
    tablero[ posicion_actual_gato[0] , posicion_actual_gato[1] ] = 2

    #imprimimos el tablero

    for fila in tablero:
        caracteres = []
        for x in fila:
            if x == 9:
                caracteres.append('🧱')
            elif x == 1:
                caracteres.append('🐁')
            elif x == 2:
                caracteres.append('🐈')
            else:
                caracteres.append('⬛')
        print(' '.join(caracteres))
    

    nueva_posicion_raton = mejor_movimiento_raton(tablero, posicion_actual_gato, posicion_actual_raton)
    posicion_actual_raton = nueva_posicion_raton

    nueva_posicion_gato = mejor_movimiento_gato(tablero, posicion_actual_gato, posicion_actual_raton)
    posicion_actual_gato = nueva_posicion_gato
    # nueva_posicion_raton = movimiento_aleatorio_raton(tablero, posicion_actual_raton)
    # posicion_actual_raton = list(nueva_posicion_raton)

    # nueva_posicion_gato = gato_persigue_linealmente(tablero, posicion_actual_gato, posicion_actual_raton)
    # if nueva_posicion_gato[0] == posicion_actual_raton[0] and nueva_posicion_gato[1] == posicion_actual_raton[1]:
        
    #     print("El gato ha ganado")

    #     break

    posicion_actual_gato = list(nueva_posicion_gato)

    #time.sleep(.5)

    


🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱
🧱 ⬛ ⬛ ⬛ ⬛ ⬛ ⬛ ⬛ ⬛ 🧱
🧱 ⬛ 🧱 ⬛ 🧱 ⬛ 🧱 ⬛ ⬛ 🧱
🧱 ⬛ 🧱 ⬛ ⬛ ⬛ 🧱 🧱 ⬛ 🧱
🧱 🧱 ⬛ 🐁 🧱 🧱 ⬛ ⬛ ⬛ 🧱
🧱 ⬛ 🧱 🧱 ⬛ ⬛ ⬛ 🧱 🐈 🧱
🧱 ⬛ ⬛ ⬛ ⬛ 🧱 ⬛ ⬛ ⬛ 🧱
🧱 🧱 ⬛ 🧱 ⬛ ⬛ ⬛ 🧱 ⬛ 🧱
🧱 ⬛ ⬛ ⬛ ⬛ 🧱 ⬛ ⬛ ⬛ 🧱
🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱 🧱


KeyboardInterrupt: 